In [80]:
from google.cloud import bigquery

class bq_jobrunner :
    
    
    def __init__(self,project_id: str,credentials_path: str,location: str):
       
        import os
        self.project_id = project_id
        self.location = location
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
        self.client = bigquery.Client()
        self.jobs = {}
        self.processed_jobs =[]
        
    def compose_query(self,query_id :int, sql: str, dest_dataset:str,
                      dest_table:str,dependent_query: list =[],common_name=''):
        job_config = bigquery.QueryJobConfig()
        job_config.destination = self.client.dataset(dest_dataset).table(dest_table)
        job_config.create_disposition = 'CREATE_IF_NEEDED'
        job_config.write_disposition = 'WRITE_TRUNCATE'
        
        job = {
            "query_id"         : query_id
            ,"sql"             : sql
            ,"job_config"      : job_config
            ,"dependent_query" : dependent_query
            ,"is_finished"     : False
            ,"common_name"     : common_name
        }
        
        self.jobs[query_id] = job

        
        
    def queue_jobs(self):
        self.queue = []
        #queue all jobs which has no dependency
        for k,v in self.jobs.items():
            dependent_queries = []
            dependent_queries = set(v['dependent_query']) - set(self.processed_jobs)
            if (v['is_finished'] == False) and (not dependent_queries):
                self.queue.append(v['query_id'])

    
    def run_job(self,job_id):
        job = self.jobs[job_id]
        print('Running "{0}"(query_id :{1}) ...'.format(job["common_name"],job["query_id"]))
        query_job = self.client.query(
            job["sql"],
            location=self.location,
            job_config=job["job_config"])
        query_job.result()
        print('Query "{}" has been finished. {:.2f} GBs are processed.'
                .format(job["common_name"],query_job.total_bytes_processed/1073741824))
        self.jobs[job_id]["is_finished"] = True
        self.processed_jobs.append(job["query_id"])
        
    def execute(self):
        from joblib import Parallel, delayed
        while len(self.jobs) != len(self.processed_jobs):
            print(len(self.processed_jobs),"jobs have been processed out of",len(self.jobs),"jobs.")
            self.queue_jobs()
            for job_id in self.queue:
                self.run_job(job_id)
            
        else:
            print("Finished all jobs.")
            
            
    def render_graph(self):
        from graphviz import Digraph
        G = Digraph(format='png')
        G.attr('node', shape='circle')

        for j in self.jobs:
            G.node(str(j), self.jobs[j]['common_name'])
            
        for j in self.jobs:
            for d in self.jobs[j]['dependent_query']:
               G.edge(str(d), str(j))

        G.render(view = True)


##TODO parallel processing

#     def execute(self):
#         from joblib import Parallel, delayed
#         while len(self.jobs) != len(self.processed_jobs):
#             print(len(self.processed_jobs),"jobs have been processed out of",len(self.jobs),"jobs.")
#             self.queue_jobs()
#             import copy
            
#             q = copy.deepcopy(self.queue)
#             Parallel(n_jobs=2, verbose=10)([delayed(MulHelper(self, 'run_job'))(job_id) for job_id in q])
            
#         else:
#             print("Finished all jobs.")
        
        
# class MulHelper(object):
#     def __init__(self, cls, mtd_name):
#         self.cls = cls
#         self.mtd_name = mtd_name

#     def __call__(self, *args, **kwargs):
#         return getattr(self.cls, self.mtd_name)(*args, **kwargs)

In [81]:
a = bq_jobrunner('sugisaki-sandbox','/Users/takuto/.credentials.json','asia-northeast1')


In [82]:
sql = '''

SELECT * FROM `sugisaki-sandbox.bq_test.TR_DELIVERY_001` 
where date between '2016-01-01' and '2017-01-01'

'''
a.compose_query(1,sql,"bq_test","query_1",common_name='2016_del')

In [83]:
sql2 = '''

SELECT * FROM `sugisaki-sandbox.bq_test.TR_DELIVERY_000` 
where date not between '2015-01-01' and '2016-01-01'

'''
a.compose_query(2,sql2,"bq_test","query_2",common_name='2015_del')

In [84]:
sql3 = '''

SELECT * FROM `sugisaki-sandbox.bq_test.query_1`

union all 

SELECT * FROM `sugisaki-sandbox.bq_test.query_2`

'''
a.compose_query(3,sql3,"bq_test","query_3",common_name='union',dependent_query=[1,2])

In [85]:
sql4 = '''

SELECT data_kbn, count(data_kbn) as cnt FROM `sugisaki-sandbox.bq_test.query_3`
group by data_kbn

'''
a.compose_query(4,sql4,"bq_test","query_4",common_name='group by',dependent_query=[3])

In [26]:
a.execute()

0 jobs have been processed out of 4 jobs.
Running "2016_del"(query_id :1) ...
Query "2016_del" has been finished. 21.29 GBs are processed.
Running "2015_del"(query_id :2) ...
Query "2015_del" has been finished. 21.26 GBs are processed.
2 jobs have been processed out of 4 jobs.
Running "union"(query_id :3) ...
Query "union" has been finished. 32.13 GBs are processed.
3 jobs have been processed out of 4 jobs.
Running "group by"(query_id :4) ...
Query "group by" has been finished. 0.42 GBs are processed.
Finished all jobs.


In [86]:
a.render_graph()